In [1]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\*\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = "/".join(pathlib.Path(x).parts[-4:-2])
        jsons[name] = json.load(f)

  0%|          | 0/92 [00:00<?, ?it/s]

In [2]:
total = len(jsons)
failed = {name for name, x in jsons.items() if x["errors"]}
total, len(failed)

(92, 23)

In [5]:
from collections import defaultdict
import re
error_templates_and_ignore = {
    "Category (.*) for object .* does not exist on spreadsheet.": False,
    "(.*) has too many vertices: .* > .*": True,
    "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.": False,
    "(.*) is missing in instance .*, so relative transform check cannot be completed.": False,
    "(.*) has bad name.": False,
    "(.*) has negative object offset scale.*": False,
    "(.*) has scale that is not 1. Reset scale.": False,
    ".*No instance ID 0 instance of (.*)": False,
    "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.": True,
    "All instances of (.*) do not have contiguous instance IDs. Missing: .*": False,
    "(.*) has disallowed type.*": False,
    "Light object (.*) should not have object offset rotation. Reset pivot.": False,
    "(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different shear. Match scaling axes on each instance.": False,
    "Model ID (.*) contains 'todo'.": False,
    "(.*) should not have an upper side.": False,
    "(.*) should have an upper side.": False,
    "(.*) has different vertex count than recorded in provider.*": False,
    "(.*) has different face count than recorded in provider.*": False,
    "(.*) has no collision mesh. Create a collision mesh.": False,
    "Inconsistent link sets within model (.*)": False,
    "(.*) is missing meta link: fillable.": False,
    "(.*) is missing meta link: heatsource.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) is missing meta link: particleapplier.": False,
    "(.*) is missing meta link: fluidsource.": False,
    "(.*) is missing meta link: fluidsink.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) element .* has too many vertices .*": True,
    "Model ID (.*) requires joints but has no joints.": False,
    "Expected meta links for (.*) are missing: .*": False,
    "Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*": False,
    "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.": False,
    "Cannot validate clothness: category (.*) not found in taxonomy.": False,
    "(.*) element .* is not a volume": True,
    "(.*) element .* has elements trimesh still finds splittable": False,
    "Object (.*) has .* fillable meshes. Should have no more than one.": False,
    "Model (.*) has category .* that is neither the from or to element in the rename file.": False,
    "(.*) has meta link not required by its synset: .*": True,
    "(.*) meta type .* ID .* has non-continuous subids .*": False,
    "Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.": False,
    "(.*) is in the deletion queue. Delete the object.": False,
    "(.*) has unapplied rename .*.": True,
    "fillable_seed is an invalid meta link under (.*).": False,
    "(.*) is not unique.": False,
    "Object (.*) has .* meshes. Should have no more than one.": False,
    "(.*) is not under another object but contains part tags {'connectedpart'}.": False,
    "(.*) is not rendering the baked material. Select the baked material for rendering or rebake.": True,
    "(.*) has non-shell material. Run texture baking.": True,
    "(.*) is not rendering the baked material in the viewport. Select the baked material for viewport.": True,
    "(.*) has different material. Match materials on each instance.": True,
    "(.*) has different UV unwrapping than recorded. Reunwrap the object.": True,
    "Non-top level material .* of (.*) is not a MultiMaterial or some kind of VRay material: .*": True,
}

SKIP_IGNORE = True

error_templates = list(error_templates_and_ignore.keys())
error_matches = defaultdict(set)
error_matches_by_file = defaultdict(lambda: defaultdict(set))
unknown_errors_by_file = defaultdict(set)
for name, f in jsons.items():
    for error in f["errors"] + f["warnings"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                if not (SKIP_IGNORE and error_templates_and_ignore[tmpl]):
                    error_matches[tmpl].add(match)
                    error_matches_by_file[tmpl][name].add(match)

                break
        else:
            print(error)
            unknown_errors_by_file[name].add(error)
        
print("\nCounts")
for tmpl, objs in sorted(error_matches.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"{tmpl}: {len(objs)}")

print("\n\nCounts by file")
for tmpl, matches_by_file in error_matches_by_file.items():
    all_matches = error_matches[tmpl]
    if not all_matches:
        continue
    print(f"\n{tmpl}")
    for name, matches in sorted(matches_by_file.items(), key=lambda x: x[1], reverse=True):
        if not matches:
            break
        print(f"  {name}: {len(matches)}")


Counts
(.*) is in the deletion queue. Delete the object.: 102
Model (.*) has category .* that is neither the from or to element in the rename file.: 22
(.*) is missing meta link: fillable.: 9
Cannot validate clothness: category (.*) not found in taxonomy.: 4
(.*) has no collision mesh. Create a collision mesh.: 3
(.*) has different vertex count than recorded in provider.*: 3
(.*) has different face count than recorded in provider.*: 3
(.*) is missing meta link: particleapplier.: 1
Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 1


Counts by file

(.*) is in the deletion queue. Delete the object.
  objects/batch-01: 2
  objects/batch-02: 1
  objects/batch-06: 2
  objects/batch-11: 1
  objects/legacy_batch-03: 1
  objects/legacy_batch-06: 1
  objects/legacy_batch-09: 2
  objects/legacy_batch-10: 1
  objects/procedural-aa: 12
  objects/task_relevant-xy: 1
  scenes/grocery_store_convenience: 55
  scenes/house_double_floor_upper: 1
  scenes/hou

In [4]:
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name
error_objs = {parse_name(x.group(1)).group("model_id") for x in error_matches["(.*) material tree contains .* with disallowed type .*. It should be one of .*."]}
print(len(error_objs))
# with open("bad_materials.json", "w") as f:
#     json.dump(sorted(error_objs), f)

0


In [7]:
from IPython.display import display, Markdown

# Print the number of kinds of error for each target
output = []
output.append("\n\n# Counts by error")
for name in jsons.keys():
    all_errors = {}
    for tmpl, matches_by_file in error_matches_by_file.items():
      if error_templates_and_ignore[tmpl]:
          continue
      matches = matches_by_file[name]
      if not matches:
          continue
      if tmpl == "(.*) is in the deletion queue. Delete the object." and "objects/" in name:
          continue
      all_errors[tmpl] = matches

    for error in unknown_errors_by_file[name]:
        all_errors["Unknown"] = all_errors.get("Unknown", set())
        all_errors["Unknown"].add(error)

    if all_errors:
        output.append(f"\n## {name}")
        for k, v in sorted(all_errors.items(), key=lambda x: len(x[1]), reverse=True):
            output.append(f"  ### {k}: {len(v)}")
            for match in v:
                err_str = match if isinstance(match, str) else match.group(0)
                output.append("    " + err_str)
            output.append("")

display(Markdown("\n".join(output)))



# Counts by error

## objects/legacy_batch-03
  ### (.*) has no collision mesh. Create a collision mesh.: 2
    dishwasher-xlmier-0-base_link has no collision mesh. Create a collision mesh.
    dishwasher-xlmier-0-link_1-base_link-R-lower-Topenable has no collision mesh. Create a collision mesh.

  ### (.*) is missing meta link: fillable.: 1
    xlmier is missing meta link: fillable.


## objects/legacy_batch-04
  ### Model (.*) has category .* that is neither the from or to element in the rename file.: 22
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.
    Model mdojox has category public_trash_can that is neither the from or to element in the rename file.
    Model pdmzhv has category public_trash_can that is neither the from or to element in the rename file.


## objects/legacy_batch-10
  ### (.*) is missing meta link: particleapplier.: 1
    njqnuo is missing meta link: particleapplier.


## objects/procedural-aa
  ### Cannot validate clothness: category (.*) not found in taxonomy.: 4
    Cannot validate clothness: category cocoa_box not found in taxonomy.
    Cannot validate clothness: category cocoa_box not found in taxonomy.
    Cannot validate clothness: category cocoa_box not found in taxonomy.
    Cannot validate clothness: category cocoa_box not found in taxonomy.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 1
    Cannot validate meta links and joints for model ID kxwgoo: Category cocoa_box not found in taxonomy.


## scenes/grocery_store_asian
  ### (.*) is missing meta link: fillable.: 1
    dtaove is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    F-grocery_shelf-dtaove-0 has no collision mesh. Create a collision mesh.


## scenes/grocery_store_convenience
  ### (.*) is in the deletion queue. Delete the object.: 55
    C-water_glass-xchqhl-0-Tglass is in the deletion queue. Delete the object.
    C-water_glass-xchqhl-1 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-1 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-0 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-15 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-3 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-13 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-14 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-5 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-11 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-12 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-10 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-9 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-ovfaus-3 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-9 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-2 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-10 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-npfpxs-4 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-7 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-8 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-8 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-npfpxs-3 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-5 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-6 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-7 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-npfpxs-2 is in the deletion queue. Delete the object.
    C-box_of_butter-brsohe-0 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-3 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-4 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-ovfaus-1 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-npfpxs-1 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-1 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-2 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-ovfaus-0 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-ovfaus-2 is in the deletion queue. Delete the object.
    C-bottle_of_beer-upmkjr-0 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-17 is in the deletion queue. Delete the object.
    C-box_of_brown_sugar-npfpxs-0 is in the deletion queue. Delete the object.
    C-box_of_butter-jgzvpv-5 is in the deletion queue. Delete the object.
    C-box_of_butter-jgzvpv-4 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-15 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-16 is in the deletion queue. Delete the object.
    C-box_of_butter-jgzvpv-2 is in the deletion queue. Delete the object.
    C-box_of_butter-jgzvpv-3 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-4 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-11 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-14 is in the deletion queue. Delete the object.
    C-box_of_butter-jgzvpv-0 is in the deletion queue. Delete the object.
    C-box_of_butter-jgzvpv-1 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-13 is in the deletion queue. Delete the object.
    C-box_of_butter-brsohe-2 is in the deletion queue. Delete the object.
    C-box_of_butter-brsohe-3 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-6 is in the deletion queue. Delete the object.
    C-bottle_of_carrot_juice-wmchsj-12 is in the deletion queue. Delete the object.
    C-box_of_butter-brsohe-1 is in the deletion queue. Delete the object.

  ### (.*) has different vertex count than recorded in provider.*: 1
    sxlklf-0 has different vertex count than recorded in provider: 184 != 154.

  ### (.*) has different face count than recorded in provider.*: 1
    sxlklf-0 has different face count than recorded in provider: 248 != 224.


## scenes/hotel_gym_spa
  ### (.*) is missing meta link: fillable.: 1
    wbnkfk is missing meta link: fillable.


## scenes/house_double_floor_upper
  ### (.*) is in the deletion queue. Delete the object.: 1
    C-centerpiece-gakixx-0 is in the deletion queue. Delete the object.


## scenes/house_single_floor
  ### (.*) is in the deletion queue. Delete the object.: 19
    garden_light-umyzpg-8 is in the deletion queue. Delete the object.
    garden_light-umyzpg-5 is in the deletion queue. Delete the object.
    garden_light-umyzpg-0 is in the deletion queue. Delete the object.
    garden_light-umyzpg-4 is in the deletion queue. Delete the object.
    garden_light-umyzpg-7 is in the deletion queue. Delete the object.
    C-statue-fwlxwe-0 is in the deletion queue. Delete the object.
    garden_light-umyzpg-3 is in the deletion queue. Delete the object.
    C-statue-iofmzi-0 is in the deletion queue. Delete the object.
    garden_light-umyzpg-10 is in the deletion queue. Delete the object.
    C-statue-denhwi-0 is in the deletion queue. Delete the object.
    garden_light-umyzpg-2 is in the deletion queue. Delete the object.
    garden_light-umyzpg-14 is in the deletion queue. Delete the object.
    garden_light-umyzpg-9 is in the deletion queue. Delete the object.
    garden_light-umyzpg-13 is in the deletion queue. Delete the object.
    garden_light-umyzpg-6 is in the deletion queue. Delete the object.
    garden_light-umyzpg-12 is in the deletion queue. Delete the object.
    garden_light-umyzpg-1 is in the deletion queue. Delete the object.
    garden_light-umyzpg-11 is in the deletion queue. Delete the object.
    C-statue-dfqibt-0 is in the deletion queue. Delete the object.

  ### (.*) is missing meta link: fillable.: 5
    kjidns is missing meta link: fillable.
    wovzuq is missing meta link: fillable.
    rvhrtr is missing meta link: fillable.
    igzoex is missing meta link: fillable.
    wtvcvz is missing meta link: fillable.


## scenes/office_large
  ### (.*) is in the deletion queue. Delete the object.: 1
    webcam-hqyxlb-0 is in the deletion queue. Delete the object.


## scenes/Rs_garden
  ### (.*) has different vertex count than recorded in provider.*: 1
    xlmier-0 has different vertex count than recorded in provider: 14257 != 2999.

  ### (.*) has different face count than recorded in provider.*: 1
    xlmier-0 has different face count than recorded in provider: 22876 != 3154.


## scenes/Rs_int
  ### (.*) has different vertex count than recorded in provider.*: 1
    xlmier-0 has different vertex count than recorded in provider: 14257 != 2999.

  ### (.*) has different face count than recorded in provider.*: 1
    xlmier-0 has different face count than recorded in provider: 22876 != 3154.


## scenes/school_computer_lab_and_infirmary
  ### (.*) is missing meta link: fillable.: 1
    wikvlp is missing meta link: fillable.


## scenes/school_gym
  ### (.*) is in the deletion queue. Delete the object.: 2
    drinking_fountain-oyipuy-1 is in the deletion queue. Delete the object.
    drinking_fountain-oyipuy-0 is in the deletion queue. Delete the object.


In [59]:
from collections import defaultdict
import re

warning_re = re.compile(r"(.*) has too many vertices: (\d+) > 20000")
warn_objs = defaultdict(dict)
for name, x in jsons.items():
    for w in x["warnings"]:
        m = warning_re.fullmatch(w)
        if m:
            warn_objs[name][m.group(1)] = int(m.group(2))
            
print(warn_objs)

defaultdict(<class 'dict'>, {'objects/batch-00': {'L-tag-ohegnp-0': 21794, 'chicken_wire-czoiqw-0': 52305}, 'objects/batch-01': {'L-shoe_rack-spdrzb-0': 59456}, 'objects/batch-02': {'L-radio-vgioak-0': 50123}, 'objects/batch-05': {'L-punching_bag-svkdji-0': 20027}, 'objects/batch-06': {'zipper-qwsvid-0': 27426}, 'objects/legacy_batch-01': {'swivel_chair-fqbnuk-0-base_link': 36405, 'swivel_chair-lizuow-0-base_link': 25743, 'swivel_chair-lxupys-0-base_link': 35465, 'swivel_chair-mqxrim-0-base_link': 21060, 'swivel_chair-opkoam-0-base_link': 38343}, 'objects/legacy_batch-02': {'table_lamp-pmekkr-0-arm-body-R-lower': 23529, 'table_lamp-pmekkr-0-head-neck-R-lower': 44331, 'table_lamp-pmekkr-0-arm-body-R-upper': 23529, 'table_lamp-pmekkr-0-head-neck-R-upper': 44331, 'table_lamp-pxdhzm-0-head-arm-R-lower': 33474, 'table_lamp-pxdhzm-0-head-arm-R-upper': 33474}, 'objects/legacy_batch-03': {'desk-egzdgz-0-base_link': 20943, 'dishwasher-znfvmj-0-link_0-base_link-P-lower': 31812, 'dishwasher-znfvm

In [46]:
# Worst offenders
sorted((cnt, obj) for warn_target in warn_objs.values() for obj, cnt in warn_target.items())

[(20082, 'barbecue_sauce_bottle-gfxrnj-0'),
 (20082, 'catsup_bottle-qfvqfm-0'),
 (20082, 'chocolate_sauce_bottle-yegrkf-0'),
 (20082, 'glaze_bottle-zdxagk-0'),
 (20082, 'hot_sauce_bottle-qvpthd-0'),
 (20082, 'mustard_bottle-lgxfyv-0'),
 (20082, 'salsa_bottle-kydilb-0'),
 (20082, 'soy_sauce_bottle-saujjl-0'),
 (20082, 'squeeze_bottle-uzmhdn-0'),
 (20082, 'sugar_syrup_bottle-kdlbbq-0'),
 (20082, 'vanilla_bottle-drevku-0'),
 (20082, 'white_sauce_bottle-gtwngf-0'),
 (20082, 'wine_sauce_bottle-vqtevv-0'),
 (20279, 'pot_plant-kxmvco-0-base_link'),
 (20943, 'desk-egzdgz-0-base_link'),
 (21060, 'swivel_chair-mqxrim-0-base_link'),
 (21794, 'L-tag-ohegnp-0'),
 (22097, 'L-garden_chair-deuron-0'),
 (22097, 'L-garden_chair-deuron-1'),
 (22560, 'room_light-fxtqtn-0'),
 (22560, 'room_light-fxtqtn-1'),
 (22740, 'fridge-jtqazu-0-base_link'),
 (22956, 'L-pedestal_table-ifmhpn-0'),
 (22956, 'L-pedestal_table-ifmhpn-1'),
 (22998, 'C-stockpot-gxiqbw-0'),
 (22998, 'C-stockpot-gxiqbw-1'),
 (23465, 'floor_lam

In [47]:
# Check which of the "articulated object has diff scale" errors are on bad objects
providers = json.loads(pathlib.Path(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json").read_text())["providers"]
id_providers = {k.split("-")[-1]: v for k, v in providers.items()}
categories = dict([tuple(reversed(k.split("-"))) for k in providers.keys()])
articulation_tmpl = "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script."
for target, objs in error_matches_by_file[articulation_tmpl].items():
    if not objs:
        continue
    bad_objs = set()
    for match in objs:
        obj = match.group(1)
        if id_providers[obj] != target:
            name = f"{categories[obj]}-{obj}"
            bad_objs.add(name)

    if not bad_objs:
        continue
    
    print(f"{target}: {len(bad_objs)} {bad_objs}")

scenes/Beechwood_0_garden: 6 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-lvgliq'}
scenes/Beechwood_0_int: 7 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Beechwood_1_int: 9 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-pluwfl', 'window-ithrgo', 'bottom_cabinet-jhymlr', 'toilet-kfmkbm', 'bottom_cabinet-kubcdk', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Benevolence_0_int: 1 {'door-lvgliq'}
scenes/Benevolence_1_int: 6 {'bottom_cabinet_no_top-ufhpbn', 'top_cabinet-dmwxyl', 'top_cabinet-eobsmt', 'top_cabinet-lsyzkh', 'window-ulnafj', 'cedar_chest-fwstpx'}
scenes/Benevolence_2_int: 2 {'window-ithrgo', 'door-lvgliq'}
scenes/hotel_gym_spa: 2 {'door-ceddpg', 'door-nfrbch'}
scenes/house_single_floor: 1 {'bottom_cabinet_no_top-gjrero'}
scenes/Ihlen_0_int: 3 {'door-ktydvs', 'win

In [48]:
# Get a list of all the objects that show up in the match links relevant errors
mismatched_link_offset_tmpl = "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*."
objects_for_match_links = error_matches[mismatched_link_offset_tmpl] | error_matches[articulation_tmpl]
print(", ".join(f'"{x.group(1)}"' for x in sorted(objects_for_match_links, key=lambda x: x.group(1))))

"bamfsz", "bamfsz", "bamfsz", "bamfsz", "bmsclc", "ceddpg", "ceddpg", "ceddpg", "dladgw", "dladgw", "dladgw", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dnyzym", "dqnbsj", "dqnbsj", "ehwmol", "ehwmol", "emeeke", "emeeke", "eobsmt", "fexqbj", "fexqbj", "fexqbj", "fqhdne", "fqhdne", "fqhdne", "fwstpx", "fwstpx", "gjrero", "gpkbiw", "gpkbiw", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "immwzb", "immwzb", "immwzb", "inmymj", "inmymj", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "jhymlr", "jhymlr", "jhymlr", "jrhgeu", "kfmkbm", "kfmkbm", "kfmkbm", "ktydvs", "ktydvs", "ktydvs", "kubcdk", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "lcrefl", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", 

In [49]:
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name

# Get a link of all the objects that are bad cloth
cloth_tmpl = "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements."
objects_for_cloth = [parse_name(x.group(1)).group('model_id') for x in error_matches[cloth_tmpl]]
print(", ".join(f'"{x}"' for x in sorted(objects_for_cloth)))

"agftpm", "aiftuk", "ajcjzr", "bgvwes", "bpjqwj", "bpjqwj", "bpjqwj", "bpjqwj", "brebdk", "crbvcg", "cuoayr", "dnypbe", "efmfdf", "ekcjci", "eknmzl", "faedft", "feohwy", "fglfga", "ficvss", "fmapwe", "fzldgi", "getwzm", "girtqm", "gtghon", "gxmcpy", "htcikj", "imdhfp", "itrkhr", "ivmsdt", "jfsovh", "jkcugp", "jmujjo", "kclcrj", "kiiium", "klhkgd", "laksie", "ldpbcu", "lqweda", "mksdlu", "mpxtgf", "mxxyva", "njidcf", "nmvvil", "nogevo", "nowqqh", "nzbhjq", "ohvomi", "ohvomi", "ojgwec", "omsbux", "padfds", "pbytey", "pchzob", "prhems", "pvzxyp", "qewdqa", "qofxar", "rcgdde", "remcyk", "rhgweu", "rygqoc", "sehjcp", "shbakk", "shbakk", "stcrad", "sullco", "szdonp", "szdonp", "szdonp", "szdonp", "tfnwti", "thmepr", "tnirgd", "trwest", "ttgike", "uaidmc", "uksirc", "uksirc", "uwzqxm", "uxgjdv", "vfknnl", "vpafgj", "vqbvph", "wbhliu", "wczokr", "xeskbh", "xzuywj", "ydgivr", "ykiiqr", "ylycdz"
